In [2]:
import pickle
import torch
import numpy as np
from torch_scatter import scatter_add
def _norm(edge_index, num_nodes, edge_weight=None, improved=False, dtype=None):
    if edge_weight is None:
        edge_weight = torch.ones((edge_index.size(1), ),
                                dtype=dtype,
                                device=edge_index.device)
    edge_weight = edge_weight.view(-1)
    assert edge_weight.size(0) == edge_index.size(1)
    row, col = edge_index.detach()
    deg = scatter_add(edge_weight.clone(), row.clone(), dim=0, dim_size=num_nodes)                                                          
    deg_inv_sqrt = deg.pow(-1)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        
    return deg_inv_sqrt, row, col

from torch_geometric.utils import add_self_loops
with open('./data/ACM/node_features.pkl','rb') as f:
        node_features = pickle.load(f)
with open('./data/ACM/edges.pkl','rb') as f:
    edges = pickle.load(f)
with open('./data/ACM/labels.pkl','rb') as f:
    labels = pickle.load(f)



/tmp/ipykernel_18788/232888377.py:23: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  edges = pickle.load(f)
/tmp/ipykernel_18788/232888377.py:23: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  edges = pickle.load(f)


In [3]:
num_nodes = edges[0].shape[0]
# build adjacency matrices for each edge type
A = []
for i,edge in enumerate(edges):
    edge_tmp = torch.from_numpy(np.vstack((edge.nonzero()[1], edge.nonzero()[0]))).type(torch.cuda.LongTensor)
    value_tmp = torch.ones(edge_tmp.shape[1]).type(torch.cuda.FloatTensor)
    # normalize each adjacency matrix
    edge_tmp, value_tmp = add_self_loops(edge_tmp, edge_attr=value_tmp, fill_value=1e-20, num_nodes=num_nodes)
    deg_inv_sqrt, deg_row, deg_col = _norm(edge_tmp.detach(), num_nodes, value_tmp.detach())
    value_tmp = deg_inv_sqrt[deg_row] * value_tmp
    A.append((edge_tmp,value_tmp))


In [4]:
edge_tmp = torch.stack((torch.arange(0,num_nodes),torch.arange(0,num_nodes))).type(torch.cuda.LongTensor)
value_tmp = torch.ones(num_nodes).type(torch.cuda.FloatTensor)
A.append((edge_tmp,value_tmp))


In [16]:
print(A)
print(A[0][0].shape)
print(A[0][1].shape)

[(tensor([[3025, 3026, 3027,  ..., 8991, 8992, 8993],
        [   0,    0,    0,  ..., 8991, 8992, 8993]], device='cuda:0'), tensor([0.3333, 0.0588, 1.0000,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0')), (tensor([[   0,  734, 3015,  ..., 8991, 8992, 8993],
        [3025, 3025, 3025,  ..., 8991, 8992, 8993]], device='cuda:0'), tensor([0.3333, 0.3333, 0.2500,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0')), (tensor([[8937, 8938, 8939,  ..., 8991, 8992, 8993],
        [   0,    1,    2,  ..., 8991, 8992, 8993]], device='cuda:0'), tensor([2.0000e-01, 3.2787e-03, 8.4034e-04,  ..., 1.0000e-20, 1.0000e-20,
        1.0000e-20], device='cuda:0')), (tensor([[   0,   75,  586,  ..., 8991, 8992, 8993],
        [8937, 8937, 8937,  ..., 8991, 8992, 8993]], device='cuda:0'), tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')), (tensor([[   0,    1,    2,  ..., 8991, 8992, 8993],
        [   0,    1,    2,  ..., 8991, 8992, 8993]], device='cuda:0'), tensor([1., 1., 1.,  ..., 1., 1., 1.], devi

In [11]:
node_features = torch.from_numpy(node_features).type(torch.cuda.FloatTensor)
print(node_features.shape)

torch.Size([8994, 1902])


In [13]:
import torch.nn.functional as F
import torch_sparse

weight = torch.Tensor(2,5)

filter = F.softmax(weight, dim=1)   
num_channels = filter.shape[0]
results = []
for i in range(num_channels):
    for j, (edge_index,edge_value) in enumerate(A):
        if j == 0:
            total_edge_index = edge_index
            total_edge_value = edge_value*filter[i][j]
        else:
            total_edge_index = torch.cat((total_edge_index, edge_index), dim=1)
            total_edge_value = torch.cat((total_edge_value, edge_value*filter[i][j]))
    
    index, value = torch_sparse.coalesce(total_edge_index.detach(), total_edge_value, m=num_nodes, n=num_nodes, op='add')
    results.append((index, value))

In [15]:
print(results[0])

(tensor([[   0,    0,    0,  ..., 8992, 8993, 8993],
        [   0, 3025, 3026,  ..., 8992, 3014, 8993]], device='cuda:0'), tensor([0.6000, 0.0667, 0.0667,  ..., 0.8000, 0.2000, 0.8000], device='cuda:0'))


In [1]:
from configparser import ConfigParser
config = ConfigParser()
filename="/home/wf/EEG_GTN/utils/data.config"
config.read(filename,encoding='UTF-8') 
print(config['path']['seed'])

/mnt/external/wf/dataset/seed


In [ ]:
exp_nums = [i for i in range(40*7)]
length=7
for i in range(40):
    exp_test = exp_nums[i*length:i*length+length]
    exp_train = [x for x in exp_nums if x not in exp_test]
    print(exp_test,exp_train)

In [13]:
import torch
data=torch.load("/home/wf/EEG_GTN/data/dataset/SEED/features_zero/seed_dcau_lds_zero/data.pt")
label=torch.load("/home/wf/EEG_GTN/data/dataset/SEED/features_zero/seed_dcau_lds_zero/label.pt")

In [14]:
print(data.shape)
print(label.shape)

torch.Size([15, 3, 15, 23, 265, 5])
torch.Size([15, 3, 15])


In [17]:
import os
root = "/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv"

features=['de','psd','dasm','dcau','rasm']

for feature in features:
    if not os.path.exists(os.path.join(root, feature)):
        os.makedirs(os.path.join(root, feature))

In [20]:
import scipy.io as sci

data=sci.loadmat("/home/wf/EEG_GTN/data/dataset/SEED_IV/eeg_feature_smooth/1/01.mat")
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'de_movingAve1', 'de_LDS1', 'psd_movingAve1', 'psd_LDS1', 'de_movingAve2', 'de_LDS2', 'psd_movingAve2', 'psd_LDS2', 'de_movingAve3', 'de_LDS3', 'psd_movingAve3', 'psd_LDS3', 'de_movingAve4', 'de_LDS4', 'psd_movingAve4', 'psd_LDS4', 'de_movingAve5', 'de_LDS5', 'psd_movingAve5', 'psd_LDS5', 'de_movingAve6', 'de_LDS6', 'psd_movingAve6', 'psd_LDS6', 'de_movingAve7', 'de_LDS7', 'psd_movingAve7', 'psd_LDS7', 'de_movingAve8', 'de_LDS8', 'psd_movingAve8', 'psd_LDS8', 'de_movingAve9', 'de_LDS9', 'psd_movingAve9', 'psd_LDS9', 'de_movingAve10', 'de_LDS10', 'psd_movingAve10', 'psd_LDS10', 'de_movingAve11', 'de_LDS11', 'psd_movingAve11', 'psd_LDS11', 'de_movingAve12', 'de_LDS12', 'psd_movingAve12', 'psd_LDS12', 'de_movingAve13', 'de_LDS13', 'psd_movingAve13', 'psd_LDS13', 'de_movingAve14', 'de_LDS14', 'psd_movingAve14', 'psd_LDS14', 'de_movingAve15', 'de_LDS15', 'psd_movingAve15', 'psd_LDS15', 'de_movingAve16', 'de_LDS16', 'psd_movingAve16', 'p

In [16]:

#coding=utf-8

import math
import torch
import torch.nn.functional as F

class Modified_SPPLayer(torch.nn.Module):

    def __init__(self, num_levels, pool_type='max_pool'):
        super(Modified_SPPLayer, self).__init__()

        self.num_levels = num_levels
        self.pool_type = pool_type
        

    def forward(self, x):
        # num:样本数量 c:通道数 h:高 w:宽
        # num: the number of samples
        # c: the number of channels
        # h: height
        # w: width
        num, c, h, w = x.size() 
#         print(x.size())
        for i in range(self.num_levels):
            level = math.pow(2,i)

            '''
            The equation is explained on the following site:
            http://www.cnblogs.com/marsggbo/p/8572846.html#autoid-0-0-0
            '''
            kernel_size = (math.ceil(h / level), math.ceil(w / level))
            stride = (math.floor(h / level), math.floor(w / level))
            pooling = (math.floor((kernel_size[0]*level-h+1)/2), math.floor((kernel_size[1]*level-w+1)/2))
            
            # update input data with padding
            zero_pad = torch.nn.ZeroPad2d((pooling[1],pooling[1],pooling[0],pooling[0]))
            x_new = zero_pad(x)
            
            # update kernel and stride
            h_new = 2*pooling[0] + h
            w_new = 2*pooling[1] + w
            
            kernel_size = (math.ceil(h_new / level), math.ceil(w_new / level))
            stride = (math.floor(h_new / level), math.floor(w_new / level))
            
            
            # 选择池化方式 
            if self.pool_type == 'max_pool':
                try:
                    tensor = F.max_pool2d(x_new, kernel_size=kernel_size, stride=stride).view(num, -1)
                except Exception as e:
                    print(str(e))
                    print(x.size())
                    print(level)
            else:
                tensor = F.avg_pool2d(x_new, kernel_size=kernel_size, stride=stride).view(num, -1)
            
            
              
            # 展开、拼接
            if (i == 0):
                x_flatten = tensor.view(num, -1)
            else:
                x_flatten = torch.cat((x_flatten, tensor.view(num, -1)), 1)
        return x_flatten


In [21]:
spp=Modified_SPPLayer(3)

x=torch.rand(12,10,10,10)
print(spp.forward(x).shape)

torch.Size([12, 210])


In [ ]:
import pandas as pd 
from scipy.spatial.distance import pdist, squareform

file_path = '/home/wf/EEG_GTN/data/dataset/SEED/channel-order.xlsx' 
data = pd.read_excel(file_path)

data.columns = ['Electrode', 'X', 'Y', 'Z']
coordinates = data[['X', 'Y', 'Z']].values

distance_matrix = squareform(pdist(coordinates))

distance_df = pd.DataFrame(distance_matrix, columns=data['Electrode'], index=data['Electrode'])

theta = 1 
tau = 0.05

def gaussian_kernel(distance, theta, tau):
    if distance <= tau:
        return np.exp(-(distance ** 2) / (2 * theta ** 2))
    else:
        return 1e-10

adjacency_matrix = distance_df.applymap(lambda x: gaussian_kernel(x, theta, tau))

import torch
torch.save(torch.from_numpy(adjacency_matrix.values).float(), "/home/wf/EEG_GTN/data/dataset/SEED/adjacency_matrix.pt")

In [2]:
import torch
torch.cuda.is_available()

True

In [8]:
import torch

data = torch.load("/root/EEG_GTN/data/dataset/Processed_Dataset/faced/zero/psd/data.pt")
label = torch.load("/root/EEG_GTN/data/dataset/Processed_Dataset/faced/zero/psd/label.pt")

In [9]:
print(data.shape,label.shape)

torch.Size([123, 1, 28, 32, 30, 5]) torch.Size([123, 1, 28])


In [10]:
data1 = data[:,:,:12,:,:,:]
data2 = data[:,:,16:,:,:,:]

label1 = label[:,:,:12]
label2 = label[:,:,16:]
print(data1.shape,label1.shape)
print(data2.shape,label2.shape)


torch.Size([123, 1, 12, 32, 30, 5]) torch.Size([123, 1, 12])
torch.Size([123, 1, 12, 32, 30, 5]) torch.Size([123, 1, 12])


In [11]:
data_new = torch.cat([data1,data2],dim=2)
label_new = torch.cat([label1,label2],dim=2)
print(data_new.shape,label_new.shape)


torch.Size([123, 1, 24, 32, 30, 5]) torch.Size([123, 1, 24])


In [12]:
torch.save(data_new,"/root/EEG_GTN/data/dataset/Processed_Dataset/faced/2class/psd/data.pt")
torch.save(label_new,"/root/EEG_GTN/data/dataset/Processed_Dataset/faced/2class/psd/label.pt")

In [7]:
import torch

label = torch.load("/root/EEG_GTN/data/dataset/Processed_Dataset/faced/2class/de/label.pt")

In [8]:
print(label)

tensor([[[0., 0., 0.,  ..., 2., 2., 2.]],

        [[0., 0., 0.,  ..., 2., 2., 2.]],

        [[0., 0., 0.,  ..., 2., 2., 2.]],

        ...,

        [[0., 0., 0.,  ..., 2., 2., 2.]],

        [[0., 0., 0.,  ..., 2., 2., 2.]],

        [[0., 0., 0.,  ..., 2., 2., 2.]]])


In [9]:
label[label == 2.] = 1.

In [10]:
print(label)

tensor([[[0., 0., 0.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 1., 1., 1.]],

        ...,

        [[0., 0., 0.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 1., 1., 1.]]])


In [11]:
torch.save(label,"/root/EEG_GTN/data/dataset/Processed_Dataset/faced/2class/de/label.pt")


In [8]:
import scipy.io as sci

data = sci.loadmat(
    "/home/wf/EEG_GTN/data/dataset/DEAP/data_preprocessed/s01.mat")
print(data['labels'].shape)

(40, 4)


In [ ]:
import scipy.io as sio
import os 

datapath = '/home/wf/EEG_GTN/data/dataset/DEAP/data_preprocessed'  # 数据路径
datapaths = os.listdir(datapath)  # 获取数据路径下的所有文件
datapaths.sort()  # 对文件名进行排序
label = torch.zeros(32, 1, 40, 4)

for idx, sub in enumerate(datapaths):

    f = open(os.path.join(datapath, sub), 'rb')

    data_sub = sio.loadmat(f)['labels']

    label[idx, 0] = torch.from_numpy(data_sub)

save_dir = os.path.join(
    "/home/wf/EEG_GTN/data/dataset/Processed_Dataset/deap/de")
torch.save(label, os.path.join(save_dir, "label.pt"))